In [1]:
import json  
import zipfile
import pandas as pd
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
d1 = None  
data = None  
with zipfile.ZipFile("train.json.zip", "r") as z:
    for filename in z.namelist():
        print(filename)
        with z.open(filename) as f:
            data = f.read()
            d1 = json.loads(data.decode("utf-8"))

train.json.json


In [3]:
# Constructing DataFrame train from a dictionary d1
train = pd.DataFrame(d1)

In [4]:
d2 = None  
data = None  
with zipfile.ZipFile("test.json.zip", "r") as z:
    for filename in z.namelist():
        print(filename)
        with z.open(filename) as f:
            data = f.read()
            d2 = json.loads(data.decode("utf-8"))

test.json


In [5]:
# Constructing DataFrame test from a dictionary d2
test = pd.DataFrame(d2)

In [6]:
print(train)

        bathrooms  bedrooms                       building_id  \
4             1.0         1  8579a0b0d54db803821a35a4a615e97a   
6             1.0         2  b8e75fc949a6cd8225b455648a951712   
9             1.0         2  cd759a988b8f23924b5a2058d5ab2b49   
10            1.5         3  53a5b119ba8f7b61d4e010512e0dfc85   
15            1.0         0  bfb9405149bfff42a92980b594c28234   
...           ...       ...                               ...   
124000        1.0         3  92bbbf38baadfde0576fc496bd41749c   
124002        1.0         2  5565db9b7cba3603834c4aa6f2950960   
124004        1.0         1  67997a128056ee1ed7d046bbb856e3c7   
124008        1.0         2  3c0574a740154806c18bdf1fddd3d966   
124009        1.0         3  d89f514c3ed0abaae52cba7017ac0701   

                    created  \
4       2016-06-16 05:55:27   
6       2016-06-01 05:44:33   
9       2016-06-14 15:19:59   
10      2016-06-24 07:54:24   
15      2016-06-28 03:50:23   
...                     ...   
12

In [7]:
# Find the words that appears 20000 times in original data
Features = pd.DataFrame(train.features.tolist()).stack().value_counts()
special = Features[Features>20000]
print(special)

Elevator           25915
Cats Allowed       23540
Hardwood Floors    23527
Dogs Allowed       22035
Doorman            20898
Dishwasher         20426
dtype: int64


In [8]:
# Insert extra fetures to training data
train.insert(0,"Elevator Available", 0)
train.insert(0,"Cats Allowed", 0)
train.insert(0,"Hardwood Floors Available", 0)
train.insert(0,"Dogs Allowed", 0)
train.insert(0,"Doorman Available", 0)
train.insert(0,"Dishwasher Available", 0)
train.features = train.features.astype(str)

In [9]:
# Add value 1 if attribute Avaliable/Allowed in original training data
train.loc[train['features'].str.contains('Elevator'), 'Elevator Available'] = 1
train.loc[train['features'].str.contains('Cats Allowed'), 'Cats Allowed'] = 1
train.loc[train['features'].str.contains('Hardwood Floors'), 'Hardwood Floors Available'] = 1
train.loc[train['features'].str.contains('Dogs Allowed'), 'Dogs Allowed'] = 1
train.loc[train['features'].str.contains('Doorman'), 'Doorman Available'] = 1
train.loc[train['features'].str.contains('Dishwasher'), 'Dishwasher Available'] = 1

In [10]:
# Insert extra fetures to testing data
test.insert(0,"Elevator Available", 0)
test.insert(0,"Cats Allowed", 0)
test.insert(0,"Hardwood Floors Available", 0)
test.insert(0,"Dogs Allowed", 0)
test.insert(0,"Doorman Available", 0)
test.insert(0,"Dishwasher Available", 0)
test.features = test.features.astype(str)

In [11]:
# Add value 1 if attribute Avaliable/Allowed in original testing data
test.loc[test['features'].str.contains('Elevator'), 'Elevator Available'] = 1
test.loc[test['features'].str.contains('Cats Allowed'), 'Cats Allowed'] = 1
test.loc[test['features'].str.contains('Hardwood Floors'), 'Hardwood Floors Available'] = 1
test.loc[test['features'].str.contains('Dogs Allowed'), 'Dogs Allowed'] = 1
test.loc[test['features'].str.contains('Doorman'), 'Doorman Available'] = 1
test.loc[test['features'].str.contains('Dishwasher'), 'Dishwasher Available'] = 1

In [12]:
# Seperate the data into training and testing sets
from sklearn.model_selection import train_test_split
# use number 0, 1, and 2 to determine 'low', 'medium' and 'high' of 'interest_level'
train['interest'] = train['interest_level'].apply(lambda t: 0 if t == 'low' else 1 if t == 'medium' else 2)

In [13]:
# Select 'bathrooms', bedrooms', 'lattice', 'longitude' and 'price' as features
X = train[['Dishwasher Available', 'Doorman Available', 'Dogs Allowed', 'Hardwood Floors Available', 'Cats Allowed','Elevator Available',
           'bathrooms', 'bedrooms', 'latitude', 'longitude', 'price'] ]
X_test = test[['Dishwasher Available', 'Doorman Available', 'Dogs Allowed', 'Hardwood Floors Available', 'Cats Allowed','Elevator Available',
           'bathrooms', 'bedrooms', 'latitude', 'longitude', 'price'] ]

# Select 'interest_level' as interest
Y = train['interest']

In [14]:
# Cross-Validation
from sklearn.model_selection import train_test_split 
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.4, random_state=1) 

In [15]:
# Implements the Naive Bayes Classifier
from sklearn.naive_bayes import GaussianNB

In [16]:
# training on training dataset 
classifier = GaussianNB()
# Fit Gaussian Naive Bayes according to X, Y training dataset
classifier.fit(X_train, Y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [17]:
# make predictions on the testing dataset
prediction = classifier.predict(X_test) 

In [18]:
print(prediction)

[1 0 1 ... 1 1 1]


In [19]:
# Make predicton of x_train and x_test
prediction1 = classifier.predict(X_train)
prediction2 = classifier.predict(X_test)

In [20]:
# Make confusion matrix
from sklearn.metrics import confusion_matrix
confusionMatrix1 = confusion_matrix(Y_train,prediction1)
confusionMatrix2 = confusion_matrix(Y_test,prediction2)

In [21]:
total1 = sum(sum(confusionMatrix1))
total2 = sum(sum(confusionMatrix2))

In [22]:
#Calculate accuracy by confusion matrix
accuracy1 = (confusionMatrix1[0,0]+confusionMatrix1[1,1])/total1
accuracy2 = (confusionMatrix2[0,0]+confusionMatrix2[1,1])/total2

In [23]:
# Print out the accuracy1
print(accuracy1)

0.2650704130221877


In [24]:
# Print out the accuracy2
print(accuracy2)

0.2641710146395826


In [25]:
# Report of Naive Bayes Classifier
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(Y_test, prediction))

              precision    recall  f1-score   support

           0       0.89      0.11      0.19     13761
           1       0.23      0.84      0.37      4441
           2       0.19      0.27      0.23      1539

    accuracy                           0.29     19741
   macro avg       0.44      0.41      0.26     19741
weighted avg       0.69      0.29      0.24     19741

